In [14]:
%pip install scikit-learn

import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [74]:
def detect_coins(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 2)

    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
        param1=100, param2=70, minRadius=40, maxRadius=120
    )

    annotated = image.copy()
    total = 0
    counts = {1:0, 5:0, 10:0, 20:0}

    if circles is not None:
        circles = np.uint16(np.around(circles[0, :]))
        circles = sorted(circles, key=lambda c: c[2], reverse=True)
        detected = []

        for (x, y, r) in circles:
            if any(np.hypot(x - dx, y - dy) < 0.8 * r for dx, dy, dr in detected):
                continue
            detected.append((x, y, r))

            x1, y1 = max(0, x-r), max(0, y-r)
            x2, y2 = min(image.shape[1], x+r), min(image.shape[0], y+r)
            coin_roi = image[y1:y2, x1:x2]

            val = None
            hsv = cv2.cvtColor(coin_roi, cv2.COLOR_BGR2HSV)

            lower_gold = np.array([10, 40, 50])
            upper_gold = np.array([40, 255, 255])
            mask_gold = cv2.inRange(hsv, lower_gold, upper_gold)

            mask = np.zeros(mask_gold.shape, dtype="uint8")
            cv2.circle(mask, (mask.shape[1]//2, mask.shape[0]//2), int(0.8*mask.shape[0]//2), 255, thickness=15)
            gold_ring = cv2.bitwise_and(mask_gold, mask)

            gold_ratio = np.sum(gold_ring > 0) / (coin_roi.shape[0] * coin_roi.shape[1])

            if gold_ratio > 0.02:
                val = 20
            else:
                if r < 55:
                    val = 1
                elif r < 70:
                    val = 5
                else:
                    val = 10

            counts[val] += 1
            total += val

            cv2.circle(annotated, (x, y), r, (0, 255, 0), 3)
            cv2.putText(annotated, f"PHP {val}", (x-30, y),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 0, 255), 2)

    return annotated, counts, total


In [75]:
def detect_bills(image, realtime=False):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    annotated = image.copy()
    total = 0
    counts = {20:0, 50:0, 100:0, 200:0, 500:0, 1000:0}

    bill_colors = {
        20:   ([5, 100, 100],   [20, 255, 255]),
        50:   ([160, 100, 100], [179, 255, 255]),
        100:  ([125, 80, 80],   [150, 255, 255]),
        200:  ([40, 80, 80],    [80, 255, 255]),
        500:  ([20, 80, 80],    [35, 255, 255]),
        1000: ([90, 80, 80],    [120, 255, 255])
    }

    for val, (lower, upper) in bill_colors.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)

        mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 30000 or area > 300000:
                continue

            rect = cv2.minAreaRect(cnt)
            (x, y), (w, h), angle = rect
            aspect_ratio = max(w, h) / (min(w, h) + 1e-5)
            if aspect_ratio < 2.0 or aspect_ratio > 3.5:
                continue

            box = cv2.boxPoints(rect)
            box = np.round(box).astype(int)
            mask_roi = np.zeros(mask.shape, dtype=np.uint8)
            cv2.drawContours(mask_roi, [box], -1, 255, -1)
            bill_area = cv2.countNonZero(mask_roi)
            color_area = cv2.countNonZero(cv2.bitwise_and(mask, mask, mask=mask_roi))

            if bill_area == 0 or color_area / bill_area < 0.3:
                continue

            counts[val] += 1
            total += val

            cv2.drawContours(annotated, [box], 0, (0, 255, 0), 3)
            cv2.putText(annotated, f"PHP {val}", (int(x)-40, int(y)),
                        cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 255), 2)

    return annotated, counts, total


In [76]:
def detect_money(image):
    annotated_coins, coin_counts, coin_total = detect_coins(image)
    annotated_bills, bill_counts, bill_total = detect_bills(annotated_coins, realtime=True)

    total = coin_total + bill_total
    counts = {**coin_counts, **bill_counts}

    cv2.putText(annotated_bills, f"TOTAL: PHP {total}", (30, 60),
                cv2.FONT_HERSHEY_TRIPLEX, 1.5, (255, 255, 255), 3)

    return annotated_bills, counts, total


In [77]:
class Activity4:
    def __init__(self, root, camera_index=0):
        self.root = root
        self.root.title("Philippine Peso Money Detector")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        controls = tk.Frame(root)
        controls.pack(pady=6)

        self.start_button = tk.Button(controls, text="Start Camera", command=self.start_camera)
        self.start_button.grid(row=0, column=0, padx=4)

        self.stop_button = tk.Button(controls, text="Stop Camera", command=self.stop_camera)
        self.stop_button.grid(row=0, column=1, padx=4)

        self.snapshot_button = tk.Button(controls, text="Save Snapshot", command=self.save_snapshot)
        self.snapshot_button.grid(row=0, column=2, padx=4)

        self.camera_index = camera_index
        self.cap = None
        self.frame = None
        self.running = False

    def start_camera(self):
        if not self.running:
            self.cap = cv2.VideoCapture(self.camera_index, cv2.CAP_DSHOW)
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

            if not self.cap.isOpened():
                print("❌ Camera not available")
                return

            self.running = True
            self.update_frame()

    def stop_camera(self):
        if self.running:
            self.running = False
            if self.cap is not None:
                self.cap.release()

    def save_snapshot(self):
        if self.frame is not None:
            cv2.imwrite("money_snapshot.png", cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB))
            print("Snapshot saved as money_snapshot.png")

    def update_frame(self):
        if self.running and self.cap is not None:
            ret, frame = self.cap.read()
            if ret:
                annotated, counts, total = detect_money(frame)
                self.frame = annotated
                img = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
        self.root.after(33, self.update_frame)


In [79]:
root = tk.Tk()
app = Activity4(root)
root.mainloop()